In [41]:
# Assignment 10

# Association Rule

In [42]:
import pandas as pd

df = pd.read_excel('Online_retail.xlsx',header=None,names=['items'])

df.head()

,items
0,"shrimp,almonds,avocado,vegetables mix,green gr..."
1,"burgers,meatballs,eggs"
2,chutney
3,"turkey,avocado"
4,"mineral water,milk,energy bar,whole wheat rice..."


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   items   7501 non-null   object
dtypes: object(1)
memory usage: 58.7+ KB


In [44]:
# converting basket to list of items

df['items'] = df['items'].apply(lambda x: x.split(','))

df.head()

,items
0,"[shrimp, almonds, avocado, vegetables mix, gre..."
1,"[burgers, meatballs, eggs]"
2,[chutney]
3,"[turkey, avocado]"
4,"[mineral water, milk, energy bar, whole wheat ..."


In [45]:
# Removing duplicates from single purchase (single basket)

df['items'] = df['items'].apply(lambda x: list(set(x)))

In [47]:
# pre-processing

from mlxtend.preprocessing import TransactionEncoder

transactions = df['items'].tolist()

encoder = TransactionEncoder()
encoded_array = encoder.fit_transform(transactions)

basket = pd.DataFrame(encoded_array, columns=encoder.columns_)

print(basket.head())

    asparagus  almonds  antioxydant juice  asparagus  avocado  babies food  \
0       False     True               True      False     True        False   
1       False    False              False      False    False        False   
2       False    False              False      False    False        False   
3       False    False              False      False     True        False   
4       False    False              False      False    False        False   

   bacon  barbecue sauce  black tea  blueberries  ...  turkey  vegetables mix  \
0  False           False      False        False  ...   False            True   
1  False           False      False        False  ...   False           False   
2  False           False      False        False  ...   False           False   
3  False           False      False        False  ...    True           False   
4  False           False      False        False  ...   False           False   

   water spray  white wine  whole weat flour

In [56]:
# metrics
# support , confidence , lift
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(basket, min_support=0.005, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

# thresholds
rules = rules[
    (rules['support'] >= 0.005) & 
    (rules['confidence'] >= 0.3) & 
    (rules['lift'] >= 1.1)
]

print(rules[['antecedents','consequents','support','confidence','lift']].head(20))

               antecedents      consequents   support  confidence      lift
0                (almonds)           (eggs)  0.006532    0.320261  1.782108
1                (almonds)  (mineral water)  0.007599    0.372549  1.562914
2                (avocado)  (mineral water)  0.011598    0.348000  1.459926
3              (black tea)  (mineral water)  0.005333    0.373832  1.568295
4                (burgers)           (eggs)  0.028796    0.330275  1.837830
5                   (cake)  (mineral water)  0.027463    0.338816  1.421397
6                (carrots)  (mineral water)  0.005333    0.347826  1.459197
7                (cereals)  (mineral water)  0.010265    0.398964  1.673729
8                (chicken)  (mineral water)  0.022797    0.380000  1.594172
9              (chocolate)  (mineral water)  0.052660    0.321400  1.348332
10          (tomato sauce)      (chocolate)  0.005066    0.358491  2.187988
11           (cooking oil)  (mineral water)  0.020131    0.394256  1.653978
12          

#### interview Questions

1. Lift measures how much more likely two items are purchased together than if they were independent (random)

   - Lift = 1 : No relationship 
   - Lift > 1 : Positive correlation
   - Lift < 1 : Negative correlation

   Confidence may look high just because an item is popular,Lift corrects for this by comparing against expected probability
   High lift rules show interesting, non-random relationships

   Marketing & Recommendations : Promote bundles of items with high lift
   Inventory : Place high-lift items together in stores

2. - Support of an itemset = proportion of transactions that contain the itemset

     Support(A) = Transactions containing A / Total Transactions
 
     Support(A >> B) = Transactions containing A and B / Total Transactions
	
     It tells us how frequent a rule/itemset is in the dataset

   - Confidence of rule (A >> B) = probability that a transaction containing A also contains B
  
     Confidence(A >> B)= Support(A) / Support(A U B)
	
     It tells how reliable the rule is.

3. - Too Many Rules : Even a medium dataset can generate thousands or millions of rules. Most of which are not useful
   - Bias Toward Frequent Items :Popular items always generate strong rules.Confidence can be misleading because it favors frequent items
   - Lack of Quantitative Information :Standard ARM treats items as binary (0/1: bought or not bought).Ignores quantity, price,...
   - No time/order information :Association rules don’t consider time/order of purchases
   - Actionability of results : Some rules are mathematically valid but not useful for business